In [13]:
!python -V

Python 3.9.21


In [14]:
import psutil

# Get available memory in GB
available_memory = psutil.virtual_memory().available / (1024 ** 3)
print(f"Available memory: {available_memory:.2f} GB")

Available memory: 4.15 GB


In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [7]:
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcap/02-experiment-tracking/mlruns/1', creation_time=1748805999299, experiment_id='1', last_update_time=1748805999299, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

EXECUTE THE FOLLOWING COMMAND INSIDE THE 02-EXPERIMENT-TRACKING FOLDER:
mlflow ui --backend-store-uri sqlite:///mlflow.db

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)[:73908]

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-02.parquet')

In [7]:
len(df_train), len(df_val)

(72258, 72212)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
# MODIFIED TO ADD MLFLOW

with mlflow.start_run():

    mlflow.set_tag("developer", "nico")

    mlflow.log_param("train-data-path", '/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param("val-data-path", '/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-01.parquet')
    
    alpha = 0.1

    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # LINE TO SAVE MODEL
    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')